In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from pyrsm import gains, gains_plot, lift, lift_plot, confusion, profit_max, ROME_max
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import copy
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

Using TensorFlow backend.


In [3]:
pentathlon_nptb= pd.read_pickle('data/pentathlon_nptb.pkl')

In [4]:
pentathlon_nptb['label']=pentathlon_nptb['buyer']=="yes"

### Probability prediction


In [5]:
categorical_columns=['gender','age']
othercol=['income','education','children',"freq_endurance","freq_strength","freq_water","freq_team","freq_backcountry", "freq_winter", "freq_racquet","total_os","training","representative"]
keep=categorical_columns+othercol+['label']+['message']
combind_data=pentathlon_nptb.loc[:,keep]
combind_data[categorical_columns] = combind_data[categorical_columns].apply(lambda x: LabelEncoder().fit_transform(x))
combind_data['message_cat']=LabelEncoder().fit_transform(combind_data['message'])

In [6]:
X_train=combind_data.loc[combind_data.training==1].drop(columns=['label','training','total_os','representative','message'])
y_train=combind_data.loc[combind_data.training==1].label
X_test=combind_data.loc[combind_data.training==0].drop(columns=['label','training','total_os','representative','message'])
y_test=combind_data.loc[combind_data.training==0].label
repre=combind_data.loc[combind_data.representative==1].drop(columns=['label','training','total_os','representative','message'])

In [7]:
Xs = np.concatenate((X_train, X_test), axis=0)

In [8]:
X_train["income_bins"] = pd.qcut(X_train["income"],10,labels=False)
X_train["education_bins"] = pd.qcut(X_train["education"],10,labels=False)
X_train["children_bins"] = pd.qcut(X_train["children"],10,labels=False,duplicates='drop')
X_test["income_bins"] = pd.qcut(X_test["income"],10,labels=False)
X_test["education_bins"] = pd.qcut(X_test["education"],10,labels=False)
X_test["children_bins"] = pd.qcut(X_test["children"],10,labels=False)

In [9]:
repre["income_bins"] = pd.qcut(repre["income"],10,labels=False,duplicates='drop')
repre["education_bins"] = pd.qcut(repre["education"],10,labels=False)
repre["children_bins"] = pd.qcut(repre["children"],10,labels=False)

In [10]:
X_train=X_train.drop(columns=['income','children','education'])
X_test=X_test.drop(columns=['income','children','education'])
repre=repre.drop(columns=['income','children','education'])

In [11]:
msg=['message','message_cat']
matchtable=combind_data.loc[:,msg].groupby('message')
matchtable=matchtable.first()

In [12]:
matchtable.sort_values(by=['message_cat'])

,message_cat
message,
backcountry,0
endurance,1
racquet,2
strength,3
team,4
water,5
winter,6


XGBoost Model

In [13]:
# Create the parameter grid: gbm_param_grid
gbm_param_grid = {
    'n_estimators': range(100,400,50),
    'max_depth': range(2,10)
}


# Instantiate the regressor: gbm
gbm = xgb.XGBClassifier(scale_pos_weight=99)

# Perform grid search: grid_auc
gbm_randomized_auc = RandomizedSearchCV(
    param_distributions=gbm_param_grid,estimator=gbm,scoring="roc_auc",n_iter=50,cv=5,verbose=1
)

# Fit grid_mse to the data
gbm_randomized_auc.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 48 is smaller than n_iter=50. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed: 111.3min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0,
                                           reg_lambda=1, scale_pos_weight=99,
                                           seed=None, silent=None, subsample=1,
                                           verbosity=1),
                   iid='d

In [14]:
print("Best parameters found: ", gbm_randomized_auc.best_params_)
print("higest auc found: ", np.abs(gbm_randomized_auc.best_score_))

Best parameters found:  {'n_estimators': 200, 'max_depth': 4}
higest auc found:  0.8874883387755101


In [15]:
#grid search result
preds =gbm_randomized_auc.predict_proba(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test.values, preds[:, 1])
auc_rf = metrics.auc(fpr, tpr)
auc_rf

0.8874828355555556

Random Forest Model

In [120]:
# Create the parameter grid: rf_param_grid
rf_param_grid = {
    'n_estimators': range(100,301,50), ### how to set n_estimators?
    'max_depth': range(2,10)
}

# Instantiate the regressor: gbm
clf = RandomForestClassifier()

# Perform grid search: grid_auc
randomized_clf = RandomizedSearchCV(
    param_distributions=rf_param_grid,estimator=clf,scoring="roc_auc",n_iter=50,cv=5,verbose=1
)

# Fit grid_mse to the data
randomized_clf.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 40 is smaller than n_iter=50. Running 40 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 18.2min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [122]:
preds =randomized_clf.predict_proba(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test.values, preds[:, 1])
auc_rf = metrics.auc(fpr, tpr)
auc_rf

0.8841045355555556

Logistic and NN model are in R markdown

#### Compare model perforance:

In [18]:
df=[["xgboost",0.8874],["random forest",0.8841],["nn",0.887],["logistic regression",0.8824]]
model_performance=pd.DataFrame(df,columns=['Model',"Auc"])

In [19]:
model_performance.sort_values(by=['Auc'],ascending=False)

,Model,Auc
2,nn,0.8875
0,xgboost,0.8874
1,random forest,0.8841
3,logistic regression,0.8824


### Profit prediction model

Model1:xgboost regression tree  

In [214]:
X_train=combind_data.loc[combind_data.training==1].drop(columns=['label','training','total_os','representative','message'])
y_train=combind_data.loc[combind_data.training==1].total_os
X_test=combind_data.loc[combind_data.training==0].drop(columns=['label','training','total_os','representative','message'])
y_test=combind_data.loc[combind_data.training==0].total_os
repre=combind_data.loc[combind_data.representative==1].drop(columns=['label','training','total_os','representative','message'])

In [215]:
DM_train = xgb.DMatrix(X_train,y_train)
DM_test =  xgb.DMatrix(X_test,y_test)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [216]:
# Create the parameter dictionary: params
from sklearn.metrics import mean_squared_error

params = {"booster":"gblinear", "objective":"reg:linear"}
xg_reg = xgb.train(params = params, dtrain=DM_train, num_boost_round=5)
preds = xg_reg.predict(DM_test)

# Compute and print the RMSE

rmse = np.sqrt(mean_squared_error(y_test,preds))

[18:42:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [217]:
rmse

46.780084269975

In [218]:
# Create the parameter grid: gbm_param_grid 
gbm_param_grid = {
    'n_estimators': [25],
    'max_depth': range(2, 12)
}

# Instantiate the regressor: gbm
gbm = xgb.XGBRegressor(n_estimators=10)

# Perform random search: grid_mse
randomized_regressor_mse = RandomizedSearchCV(param_distributions=gbm_param_grid,estimator=gbm,scoring="neg_mean_squared_error",n_iter=5,cv=4,verbose=1)


# Fit randomized_mse to the data
randomized_regressor_mse.fit(X_train,y_train)

# Print the best parameters and lowest RMSE
#print("Best parameters found: ", randomized_regressor_mse.best_params_)
#print("Lowest RMSE found: ", np.sqrt(np.abs(randomized_regressor_mse.best_score_)))

Fitting 4 folds for each of 5 candidates, totalling 20 fits
[18:43:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[18:43:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[18:43:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[18:44:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   46.5s finished
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[18:44:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=4, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=10,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_state=0, reg_alpha=0,
                                          reg_lambda=1, scale_pos_weight=1,
                                          seed=None, silent=None, subsample=1,
                                   

In [219]:
pred_ords=randomized_regressor_mse.predict(X_test)

In [221]:
rmse = np.sqrt(mean_squared_error(y_test,pred_ords))

In [222]:
rmse

46.07328966080632

Model2: MLP

In [252]:
## standarlization
X_train=combind_data.loc[combind_data.training==1].drop(columns=['label','training','total_os','representative','message'])
y_train=combind_data.loc[combind_data.training==1].total_os
X_test=combind_data.loc[combind_data.training==0].drop(columns=['label','training','total_os','representative','message'])
y_test=combind_data.loc[combind_data.training==0].total_os
repre=combind_data.loc[combind_data.representative==1].drop(columns=['label','training','total_os','representative','message'])
to_standard=['income','education','children',"freq_endurance","freq_strength","freq_water","freq_team","freq_backcountry", "freq_winter", "freq_racquet"]

standardizer=StandardScaler()
standardizer.fit(X_train[to_standard])
X_train[to_standard] = standardizer.transform(X_train[to_standard])
X_test[to_standard] = standardizer.transform(X_test[to_standard])
repre[to_standard] = standardizer.transform(repre[to_standard])

In [254]:
# define the keras model. Starting from 1 hidden layer
model1 = Sequential()
model1.add(Dense(128,activation='relu',input_dim=X_train.shape[1]))
model1.add(Dense(1))
early_stopping_monitor = EarlyStopping(patience=4)
model1.compile(loss='mse', optimizer='adam', metrics=['mse'])
model1_training=model1.fit(X_train, y_train, epochs=50,callbacks=[early_stopping_monitor], verbose=False,validation_split=0.2)
model1_training.history['val_loss']

[1925.2600703125,
 1911.7102521972656,
 1925.9681008649554,
 1919.0705687081472,
 1908.049925641741,
 1902.6662305385044,
 1900.5928201032366,
 1904.5289628208704,
 1898.2716292550224,
 1898.1073904854911,
 1902.8352825753348,
 1896.7106726771763,
 1899.9355705217633,
 1895.1287940848215,
 1906.9401216517856,
 1896.3939266183036,
 1894.6496606096541,
 1898.182575265067,
 1896.1091954520089,
 1933.8024420340403,
 1896.408396344866]

In [239]:
# define the keras model. Starting from 1 hidden layer
model = Sequential()
model.add(Dense(128,activation='relu',input_dim=X_train.shape[1]))
model.add(Dense(128,activation='relu'))
model.add(Dense(1))
early_stopping_monitor = EarlyStopping(patience=4)
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model2_training=model.fit(X_train, y_train, epochs=50,callbacks=[early_stopping_monitor], verbose=False,validation_split=0.2)
model2_training.history['val_loss']

[1910.4122819475447,
 1904.9750271344867,
 1905.2412004394532,
 1911.1469476492746,
 1909.8916578543526,
 1906.86596648298]

In [241]:
# define the keras model. Starting from 1 hidden layer
model = Sequential()
model.add(Dense(64,activation='relu',input_dim=X_train.shape[1]))
model.add(Dense(1))
early_stopping_monitor = EarlyStopping(patience=4)
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model3_training=model.fit(X_train, y_train, epochs=50,callbacks=[early_stopping_monitor], verbose=False,validation_split=0.2)
model3_training.history['val_loss']

[1930.1978723493303,
 1920.1233324497769,
 1906.7807376185826,
 1904.060255092076,
 1906.1430611049107,
 1903.540924734933,
 1901.4868044084822,
 1897.7071743164063,
 1900.6204453125,
 1903.7933172084263,
 1898.2711911272322,
 1896.5529031808035,
 1896.878449951172,
 1896.983378731864,
 1909.258444719587,
 1898.0410813685826]

In [255]:
pred_ords=model1.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test,pred_ords))

In [256]:
rmse

45.73590125073705

Model3:linear regression (in R)

Model4:Random forest (in R)

Compare model performance:

In [22]:
regression_model=[["xgboost",46.07],["MLP",45.73],["linear regression",46.00],["random forest",46.21],["nn",45.83]]
model_performance=pd.DataFrame(regression_model,columns=['Model',"RMSE"])
model_performance.sort_values(by=['RMSE'],ascending=True)

,Model,RMSE
1,MLP,45.73
4,nn,45.83
2,linear regression,46.00
0,xgboost,46.07
3,random forest,46.21


#### Because the mlp model yielded the lowest RMSE,we use this model to predict order size.

In [258]:
# assign a new column and fill with 7 products and then calculate if message this product,the total_ords

repre_backcountry=repre.assign(message_cat=0)
backcountry_profit=model1.predict(repre_backcountry)*0.4
repre_endurance=repre.assign(message_cat=1)
endurance_profit=model1.predict(repre_endurance)*0.4

repre_racquet=repre.assign(message_cat=2)
racquet_profit=model1.predict(repre_racquet)*0.4

repre_strength=repre.assign(message_cat=3)
strength_profit=model1.predict(repre_strength)*0.4

repre_team=repre.assign(message_cat=4)
team_profit=model1.predict(repre_team)*0.4

repre_water=repre.assign(message_cat=5)
water_profit=model1.predict(repre_water)*0.4

repre_winter=repre.assign(message_cat=6)
winter_profit=model1.predict(repre_winter)*0.4


In [293]:
# calculate the maximum profit and return the product(column name)
# add the profit to dataframe
repre_profit=repre.assign(backcountry=backcountry_profit,endurance=endurance_profit,
                        racquet=racquet_profit,strength=strength_profit,
                        team=team_profit,water=water_profit,winter=winter_profit)

In [294]:
repre_profit.to_csv("repre_profit_deeplearning.csv")